# Tokenization

> Load and configure the pre-trained tokenizer (e.g., ai4bharat/indic-bert).

In [ ]:
#| default_exp data.tokenization

## Colab Setup

In [ ]:
#| hide
# Mount Google Drive (Optional, but recommended for persistent storage)
from pathlib import Path

try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ModuleNotFoundError:
    print("Not running in Colab, skipping Drive mount.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

Mounted at /content/drive
Google Drive mounted successfully.


In [ ]:
#| export
try:
    import indic_clip.core
    print("Reloaded indic_clip.core")
except ModuleNotFoundError:
    print("indic_clip.core not found initially.")
    # Attempt to set sys.path if running in Colab and project cloned
    import sys
    if 'google.colab' in sys.modules:
        project_parent = '/content' # Assuming cloned into /content/indic-clip
        if Path('/content/drive/MyDrive/Indic-Clip').exists():
             project_parent = '/content/drive/MyDrive/Indic-Clip'
        if project_parent not in sys.path:
             sys.path.insert(0, project_parent)
             print(f"Added {project_parent} to sys.path")
        try:
            import indic_clip.core
            print("Imported indic_clip.core after path adjustment.")
        except ModuleNotFoundError:
            print("ERROR: Still cannot find indic_clip.core. Ensure project structure is correct.")
            print("Expected: /content/Indic-Clip/indic_clip/core.py or similar in Drive")
            # raise # Stop execution if core components missing

indic_clip.core not found initially.
Added /content/drive/MyDrive/Indic-Clip to sys.path
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive detected, setting PROJECT_ROOT to /content/drive/MyDrive/Indic-Clip
Ensure your project files are located there.
Imported indic_clip.core after path adjustment.


In [ ]:
#| hide
# Install necessary libraries if in Colab
import sys
if 'google.colab' in sys.modules:
    !pip install -q nbdev transformers torch sentencepiece # Sentencepiece might be a dependency for some tokenizers
    # Ensure indic_clip core is accessible (adjust path if needed)
    # !pip install -e .

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import logging
from transformers import AutoTokenizer
from pathlib import Path
import torch

try:
    from indic_clip.core import (
        PRETRAINED_TOKENIZER_NAME,
        TOKENIZER_PATH,
        CUSTOM_SPECIAL_TOKENS,
        CLS_TOKEN, SEP_TOKEN, PAD_TOKEN, UNK_TOKEN, # Import standard tokens for reference
        get_logger,
        setup_logging,
        ensure_dir
    )
except ModuleNotFoundError:
    print("Error importing from indic_clip.core. Using Fallbacks.")
    # Fallbacks if core isn't importable (e.g., interactive testing)
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__)
    PRETRAINED_TOKENIZER_NAME = "ai4bharat/indic-bert"
    TOKENIZER_PATH = Path('./models/tokenizer')
    CUSTOM_SPECIAL_TOKENS = ["<Sa>", "<Hi>"]
    CLS_TOKEN, SEP_TOKEN, PAD_TOKEN, UNK_TOKEN = "[CLS]", "[SEP]", "[PAD]", "[UNK]"
    def get_logger(name): return logging.getLogger(name)
    def setup_logging(): pass
    def ensure_dir(path: Path): path.mkdir(parents=True, exist_ok=True)


# Setup logging
setup_logging()
logger = get_logger(__name__)

## Load Pre-trained Tokenizer

In [ ]:
#| export
class IndicBERTTokenizer:
    """Wrapper for the Hugging Face tokenizer (ai4bharat/indic-bert)."""
    def __init__(self, tokenizer_name: str = PRETRAINED_TOKENIZER_NAME, max_length: int = 77):
        """
        Initializes the wrapper by loading the pre-trained tokenizer.

        Args:
            tokenizer_name (str): The name of the Hugging Face tokenizer model.
            max_length (int): The maximum sequence length for padding/truncation.
        """
        self.tokenizer_name = tokenizer_name
        self.max_length = max_length
        try:
            self.hf_tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
            logger.info(f"Successfully loaded tokenizer: {tokenizer_name}")
            self._add_custom_special_tokens()
        except Exception as e:
            logger.error(f"Failed to load tokenizer '{tokenizer_name}': {e}")
            raise

    def _add_custom_special_tokens(self):
        """Adds custom special tokens if they don't already exist."""
        if not CUSTOM_SPECIAL_TOKENS:
            return

        num_added_toks = self.hf_tokenizer.add_special_tokens({'additional_special_tokens': CUSTOM_SPECIAL_TOKENS})
        if num_added_toks > 0:
            logger.info(f"Added {num_added_toks} custom special tokens: {CUSTOM_SPECIAL_TOKENS}")
            # Note: The text encoder's embedding layer needs to be resized after adding tokens
            # model.resize_token_embeddings(len(tokenizer.hf_tokenizer))
            logger.warning("Remember to resize model token embeddings if new tokens were added!")
        else:
            logger.info("Custom special tokens already exist or none were specified.")

    def tokenize(self, texts: list[str] | str) -> dict:
        """Tokenizes a single text or a batch of texts.

        Args:
            texts (list[str] | str): The text or list of texts to tokenize.

        Returns:
            dict: A dictionary containing 'input_ids' and 'attention_mask' tensors.
        """
        # Handle single string input
        if isinstance(texts, str):
            texts = [texts]

        # Tokenize using the Hugging Face tokenizer
        # It handles adding CLS/SEP based on its configuration
        # padding='max_length' ensures all sequences have the same length
        # truncation=True cuts sequences longer than max_length
        # return_tensors='pt' returns PyTorch tensors
        try:
            batch_encodings = self.hf_tokenizer(
                texts,
                padding='max_length',
                truncation=True,
                max_length=self.max_length,
                return_tensors='pt'
            )
            return batch_encodings
        except Exception as e:
            logger.error(f"Error during tokenization: {e}")
            # Return empty tensors or raise error based on desired handling
            return {'input_ids': torch.tensor([]), 'attention_mask': torch.tensor([])}

    def decode(self, token_ids: torch.Tensor | list[int]) -> list[str] | str:
        """Decodes token IDs back into text.

        Args:
            token_ids (torch.Tensor | list[int]): A tensor or list of token IDs.
                                                  Can be a single sequence or a batch.

        Returns:
            list[str] | str: The decoded text(s).
        """
        try:
            # Handle tensor input
            if isinstance(token_ids, torch.Tensor):
                 # If it's a batch, decode each sequence
                if token_ids.ndim > 1:
                     return self.hf_tokenizer.batch_decode(token_ids, skip_special_tokens=True)
                else:
                     return self.hf_tokenizer.decode(token_ids, skip_special_tokens=True)
            # Handle list input (assume single sequence)
            elif isinstance(token_ids, list):
                 return self.hf_tokenizer.decode(token_ids, skip_special_tokens=True)
            else:
                 logger.error(f"Unsupported input type for decode: {type(token_ids)}")
                 return ""
        except Exception as e:
            logger.error(f"Error during decoding: {e}")
            return ""

    @property
    def vocab_size(self) -> int:
        """Returns the size of the tokenizer's vocabulary."""
        return len(self.hf_tokenizer)

    @property
    def special_tokens(self) -> dict:
        """Returns a dictionary of special tokens used by the tokenizer."""
        return self.hf_tokenizer.special_tokens_map

    @property
    def pad_token_id(self) -> int | None:
        """Returns the ID used for padding."""
        return self.hf_tokenizer.pad_token_id

    def save_tokenizer(self, save_directory: Path = TOKENIZER_PATH):
        """Saves the tokenizer's state (including vocab and added tokens) to a directory."""
        try:
            ensure_dir(save_directory)
            self.hf_tokenizer.save_pretrained(str(save_directory))
            logger.info(f"Tokenizer state saved to {save_directory}")
        except Exception as e:
            logger.error(f"Failed to save tokenizer state to {save_directory}: {e}")

    @classmethod
    def load_tokenizer(cls, load_directory: Path = TOKENIZER_PATH, max_length: int = 77):
        """Loads tokenizer state from a directory."""
        if not load_directory.exists():
             logger.error(f"Cannot load tokenizer. Directory not found: {load_directory}")
             raise FileNotFoundError(f"Tokenizer directory not found: {load_directory}")
        try:
            # Instantiate the class, which loads the tokenizer from the specified dir
            instance = cls(tokenizer_name=str(load_directory), max_length=max_length)
            logger.info(f"Tokenizer state loaded successfully from {load_directory}")
            return instance
        except Exception as e:
            logger.error(f"Failed to load tokenizer state from {load_directory}: {e}")
            raise

## Example Usage

In [ ]:
#| eval: false
if __name__ == '__main__':
    print(f"Loading tokenizer: {PRETRAINED_TOKENIZER_NAME}")
    try:
        tokenizer = IndicBERTTokenizer(max_length=64)

        print(f"Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")
        print(f"Special tokens map: {tokenizer.special_tokens}")
        print(f"PAD token ID: {tokenizer.pad_token_id}")
        print(f"Custom Special Tokens potentially added: {CUSTOM_SPECIAL_TOKENS}")

        # Example tokenization
        hindi_text = "गुलाबी पोशाक में बच्चा प्रवेश के रास्ते में सीढ़ियों के सेट पर चढ़ रहा है।"
        sanskrit_text = "कृष्णं वन्दे जगद्गुरुम्।" # Example
        texts_to_tokenize = [hindi_text, sanskrit_text, "A short sentence."]

        print(f"\nTokenizing example texts (max_length={tokenizer.max_length}):")
        batch_encoding = tokenizer.tokenize(texts_to_tokenize)

        print("Input IDs:")
        print(batch_encoding['input_ids'])
        print("\nAttention Mask:")
        print(batch_encoding['attention_mask'])

        # Example decoding
        print("\nDecoding the first sequence:")
        decoded_text = tokenizer.decode(batch_encoding['input_ids'][0])
        print(f"Decoded: '{decoded_text}'")

        print("\nDecoding the batch:")
        decoded_batch = tokenizer.decode(batch_encoding['input_ids'])
        print(decoded_batch)

        # Example save/load
        print(f"\nSaving tokenizer state to: {TOKENIZER_PATH}")
        tokenizer.save_tokenizer()

        print("\nLoading tokenizer state from save directory...")
        loaded_tokenizer = IndicBERTTokenizer.load_tokenizer(max_length=64)
        print(f"Loaded tokenizer vocab size: {loaded_tokenizer.vocab_size}")
        # Verify consistency
        assert tokenizer.vocab_size == loaded_tokenizer.vocab_size
        re_encoded = loaded_tokenizer.tokenize(texts_to_tokenize)
        assert torch.equal(batch_encoding['input_ids'], re_encoded['input_ids'])
        print("Save/Load test passed.")

    except Exception as e:
        print(f"An error occurred in the example usage: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
#| hide
%cd /content/drive/MyDrive/Indic-Clip

/content/drive/MyDrive/Indic-Clip


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export() # Run this in terminal to export